In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")

## LOAD DATA

In [ ]:
covid = pd.read_parquet('covidClean.parquet')

toRemove = ["PATIENT_ID", "USMER", "SYMPTOMS_DATE",
            "MEDICAL_UNIT", "ADMISSION_DATE", "PATIENT_TYPE",
            "DEATH_DATE", "ORIGIN_COUNTRY"]
covid = covid.drop(columns = toRemove)
covid = covid.drop(columns= ["DIED", "INTUBED", "ICU"])


covid.AGE = covid.AGE.astype('float32')
print(covid.info())

In [ ]:
arr = np.array(covid.drop(columns = ["AT_RISK"]).values, dtype=np.float32)

X = torch.tensor(arr).to(device)
Y = torch.tensor(covid["AT_RISK"].values, dtype=torch.float32).reshape(-1, 1).to(device)

In [ ]:

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(14, 32)
        self.actInput = nn.ReLU()
        self.hidden1 = nn.Linear(32, 32)
        self.act1 = nn.ReLU()
        self.output = nn.Linear(32, 1)
        self.actOutput = nn.Sigmoid()
        
    def forward(self, x):
        x = self.actInput(self.input(x))
        x = self.act1(self.hidden1(x))
        x = self.actOutput(self.output(x))
        return x
    
mlp = MLP().to(device)
print(mlp)

In [ ]:
lossFN = nn.BCELoss().to(device)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)

In [ ]:
n_epoch = 100
batchSize = 32

for epoch in range(n_epoch):
     loss = None
     for i in range(0, len(X), batchSize):
          x = X[i:i+batchSize]
          y = Y[i:i+batchSize]
          pred = mlp(x)
          loss = lossFN(pred, y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
     print(f"Epoch {epoch} loss: {loss}")